In [1]:
from dumbcoin import Blockchain
import time

In [2]:
# Initialize a blockchain with 2000 initial coins
dumbcoin = Blockchain(2000)

Genesis block mined in 0.3972729999986768s


In [3]:
# Print the blockchain and confirm that it is valid
print(dumbcoin)
print(dumbcoin.verify_blockchain())

Block: 0,
Time: 1516399579.3628569,
Proof: 167701,
Transactions: {'amount': 2000, 'sender': None, 'timestamp': 1516399579.3628569, 'recipient': 'genesis'}

True


In [4]:
# Set up some initial transactions
dumbcoin.add_transaction("genesis", "adam", 1000)
dumbcoin.add_transaction("genesis", "eve", 1000)
# And add them into a new block
dumbcoin.add_block()

# And some more giving
dumbcoin.add_transaction("adam", "cain", 100)
dumbcoin.add_transaction("adam", "abel", 200)
dumbcoin.add_block()

Staging transaction: {'amount': 1000, 'sender': 'genesis', 'timestamp': 1516399580.2785423, 'recipient': 'adam'}
Validating transaction: {'amount': 1000, 'sender': 'genesis', 'timestamp': 1516399580.2785423, 'recipient': 'adam'}
Transaction validated against ledger
Transaction stagged. 1 transactions awaiting mining
Staging transaction: {'amount': 1000, 'sender': 'genesis', 'timestamp': 1516399580.27939, 'recipient': 'eve'}
Validating transaction: {'amount': 1000, 'sender': 'genesis', 'timestamp': 1516399580.27939, 'recipient': 'eve'}
Transaction validated against ledger
Transaction stagged. 2 transactions awaiting mining
Mining new block with 2 transactions
New block at index 1 mined in 0.12091999999938707s
Block successfully added to blockchain at index 1
Staging transaction: {'amount': 100, 'sender': 'adam', 'timestamp': 1516399580.4013183, 'recipient': 'cain'}
Validating transaction: {'amount': 100, 'sender': 'adam', 'timestamp': 1516399580.4013183, 'recipient': 'cain'}
Transaction

In [6]:
# Print and verify the new chain
print(dumbcoin)
print(dumbcoin.verify_blockchain())

Block: 2,
Time: 1516399580.4023788,
Proof: 156707,
Transactions: {'amount': 100, 'sender': 'adam', 'timestamp': 1516399580.4013183, 'recipient': 'cain'}
{'amount': 200, 'sender': 'adam', 'timestamp': 1516399580.4018154, 'recipient': 'abel'}

Block: 1,
Time: 1516399580.2799523,
Proof: 40733,
Transactions: {'amount': 1000, 'sender': 'genesis', 'timestamp': 1516399580.2785423, 'recipient': 'adam'}
{'amount': 1000, 'sender': 'genesis', 'timestamp': 1516399580.27939, 'recipient': 'eve'}

Block: 0,
Time: 1516399579.3628569,
Proof: 167701,
Transactions: {'amount': 2000, 'sender': None, 'timestamp': 1516399579.3628569, 'recipient': 'genesis'}

True


In [7]:
transactions = dumbcoin.get_settled_transactions()
for transaction in transactions:
    print(transaction)

{'amount': 2000, 'sender': None, 'timestamp': 1516399579.3628569, 'recipient': 'genesis'}
{'amount': 1000, 'sender': 'genesis', 'timestamp': 1516399580.2785423, 'recipient': 'adam'}
{'amount': 1000, 'sender': 'genesis', 'timestamp': 1516399580.27939, 'recipient': 'eve'}
{'amount': 100, 'sender': 'adam', 'timestamp': 1516399580.4013183, 'recipient': 'cain'}
{'amount': 200, 'sender': 'adam', 'timestamp': 1516399580.4018154, 'recipient': 'abel'}


In [8]:
len(dumbcoin)

3

In [10]:
ledger = dumbcoin.create_ledger(transactions)
print(ledger)
valid_trans = {"sender": "adam", "recipient": "eve", "amount": 200, "timestamp": time.time()}
print(dumbcoin.validate_transaction(valid_trans, transactions))
invalid_trans = {"sender": "adam", "recipient": "cain", "amount": 20000, "timestamp": time.time()}
print(dumbcoin.validate_transaction(invalid_trans, transactions))
really_invalid_trans = {"sender": "lucifer", "recipient": "cain", "amount": 700, "timestamp": time.time()}
print(dumbcoin.validate_transaction(really_invalid_trans, transactions))

{'genesis': 0, 'abel': 200, 'cain': 100, 'adam': 700, 'eve': 1000}
Validating transaction: {'amount': 200, 'sender': 'adam', 'timestamp': 1516399639.8680868, 'recipient': 'eve'}
True
Validating transaction: {'amount': 20000, 'sender': 'adam', 'timestamp': 1516399639.8684483, 'recipient': 'cain'}
False
Validating transaction: {'amount': 700, 'sender': 'lucifer', 'timestamp': 1516399639.86877, 'recipient': 'cain'}
False
